In [8]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None 

### Nous ferons ici une analyse économétrique des données comme vu dans les cours de l'ensae. 

In [21]:
file_path=r'..\..\Recuperation_data\France_Travail\offres_emploi_concatene_cleaned.csv'
data = pd.read_csv(r'..\..\Recuperation_data\France_Travail\offres_emploi_concatene_cleaned.csv')
data.head()

,intitule,source,departement,salaire_annuel,etude_requise,experience_requise,contrat,romeCode_extract,tps_Pp,secteur_activite,qualificationCode
0,Administrateur systèmes et réseaux informatiqu...,Informatique,41.0,38000.0,5.0,24.0,2,M18,1.0,78.0,8.0
1,Technicien informatique F/H,Informatique,69.0,24238.0,NaN,12.0,2,H11,1.0,78.0,6.0
2,Ingénieur de production/exploitation informati...,Informatique,41.0,35000.0,5.0,60.0,2,M18,1.0,78.0,9.0
3,Technicien informatique (H/F),Informatique,NaN,27000.0,5.0,0.0,0,I14,1.0,66.0,5.0
4,Chef de projet réseau informatique (H/F),Informatique,74.0,45000.0,NaN,132.0,0,M18,1.0,78.0,2.0


### Regression sur les études et l'experiance

In [22]:
data_exp_etude = data[['salaire_annuel', 'etude_requise' , 'experience_requise']]

data_exp_etude['salaire_annuel'] = pd.to_numeric(data_exp_etude['salaire_annuel'], errors='coerce')
data_exp_etude['etude_requise'] = pd.to_numeric(data_exp_etude['etude_requise'], errors='coerce')
data_exp_etude['experience_requise'] = pd.to_numeric(data_exp_etude['experience_requise'], errors='coerce')

# Supprimer les lignes contenant des valeurs manquantes
data_exp_etude = data_exp_etude.dropna(axis=0, how='any')

In [23]:
X = data_exp_etude[['etude_requise' , 'experience_requise']]
y = data_exp_etude[['salaire_annuel']]

# Initialisation et entraînement du modèle
model = LinearRegression()
model.fit(X, y)

X_with_const = sm.add_constant(X)  # Ajout de l'interception
model_stats = sm.OLS(y, X_with_const)  # Création du modèle
results = model_stats.fit()  # Ajustement du modèle

# Résumé détaillé
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         salaire_annuel   R-squared:                       0.353
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     3160.
Date:                Sun, 29 Dec 2024   Prob (F-statistic):               0.00
Time:                        21:28:59   Log-Likelihood:            -1.2166e+05
No. Observations:               11561   AIC:                         2.433e+05
Df Residuals:                   11558   BIC:                         2.433e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               1.464e+04    259

Les résultats de la régression montrent que le niveau d'études requis et l'expérience exigée ont un impact significatif sur le salaire annuel. Le modèle explique 35,2 % de la variance du salaire, avec des coefficients positifs pour les deux variables : chaque niveau d'études supplémentaire augmente le salaire de 2 644,69 €, et chaque année d'expérience ajoutée augmente le salaire de 160,59 €. Les tests statistiques montrent que ces effets sont statistiquement significatifs. 

### Regression général

In [24]:
data_reg = data[['salaire_annuel', 'etude_requise' , 'experience_requise', 'source','departement','contrat','secteur_activite']]

data_reg['salaire_annuel'] = pd.to_numeric(data_reg['salaire_annuel'], errors='coerce')
data_reg['etude_requise'] = pd.to_numeric(data_reg['etude_requise'], errors='coerce')
data_reg['experience_requise'] = pd.to_numeric(data_reg['experience_requise'], errors='coerce')

data_reg = pd.get_dummies(data_reg, columns=['source','departement','contrat','secteur_activite'], dtype=int)
# Supprimer les lignes contenant des valeurs manquantes
data_reg= data_reg.dropna(axis=0, how='any')

In [25]:
cols = data_reg.columns.tolist()
cols.remove('salaire_annuel')

X = data_reg[cols]
y = data_reg[['salaire_annuel']]

# Initialisation et entraînement du modèle
model = LinearRegression()
model.fit(X, y)

X_with_const = sm.add_constant(X)  # Ajout de l'interception
model_stats = sm.OLS(y, X_with_const)  # Création du modèle
results = model_stats.fit()  # Ajustement du modèle

with open("OLS_results_summary.txt", "w") as file:
    file.write(results.summary().as_text())

# Partie prédictive

In [26]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Création du modèle Lasso
alpha_value = 0.1  # Force de la régularisation
lasso_model = Lasso(alpha=alpha_value)

# Entraînement du modèle
lasso_model.fit(X_train, y_train)

# Prédictions
y_pred = lasso_model.predict(X_test)

# Affichage des coefficients
print("Coefficients du modèle :", lasso_model.coef_)
print("Biais (intercept) :", lasso_model.intercept_)

Coefficients du modèle : [ 2.04995538e+03  1.46114963e+02 -1.19855050e+03 -2.54558214e+03
  4.03561316e+03 -3.79772629e+03 -1.25380949e+03  6.09140126e+02
  5.73785641e+03 -1.47038262e+03 -8.04241839e+02  2.88887535e+02
  8.90491789e+02  6.26781370e+02  6.47011679e+02  7.85766783e+02
 -1.14165296e+03  7.56145867e+02 -5.51157975e+02 -1.17370852e+03
 -8.04458504e+03  0.00000000e+00 -9.91623111e+03 -2.15628308e+03
 -5.97548644e+03 -6.56750248e+03 -4.89411930e+03 -6.53720115e+03
  0.00000000e+00 -8.67417680e+03 -5.31939768e+03 -7.35465682e+03
 -5.83582994e+03 -7.00975140e+03 -8.32272213e+03 -5.87134390e+03
 -8.07387103e+03 -6.16496439e+03 -7.46451345e+03 -6.78465326e+03
 -6.48876839e+03 -7.84141359e+03 -6.97163451e+03 -5.66668548e+03
 -6.97424463e+03 -5.91626110e+03 -7.32453687e+03 -7.53558463e+03
 -7.80065559e+03 -8.83786638e+03 -6.97078146e+03 -5.51398934e+03
 -6.04516437e+03 -6.65730226e+03 -6.00757942e+03 -8.34953161e+03
 -4.66050760e+03 -6.21869231e+03 -7.00959399e+03 -5.99768618e+03


c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.012e+10, tolerance: 1.079e+08
  model = cd_fast.enet_coordinate_descent(


In [27]:
# Calcul du Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((np.array(y_test) - y_pred) / np.array(y_test)).flatten()) * 100
print("Mean Absolute Percentage Error (MAPE):", mape)

Mean Absolute Percentage Error (MAPE): 33.18043679065281
